In [68]:
#so we are bulding separate models
#importing basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams
import seaborn as sns

#machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LassoCV,Lasso
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.ensemble import GradientBoostingRegressor
 

In [ ]:
dat = pd.read_csv(r'C:\Users\Aditya\Desktop\CS data competition\Imputed.csv')

In [3]:
#filtering for sj

sj = dat[dat['city']=='sj']

In [38]:
#removing obvious variable due to high correlation with other variables, OR low correlation with target variable
var_remove = ['ndvi_ne_x','ndvi_nw_x','ndvi_se_x','ndvi_sw_x','reanalysis_air_temp_k_x','reanalysis_avg_temp_k_x','station_avg_temp_c_x','station_max_temp_c_x','station_min_temp_c_x']

In [39]:
sj_new = sj.drop(var_remove,axis=1)

In [40]:
sj_new.head()

,city,year,weekofyear,week_start_date,precipitation_amt_mm_x,reanalysis_dew_point_temp_k_x,reanalysis_max_air_temp_k_x,reanalysis_min_air_temp_k_x,reanalysis_precip_amt_kg_per_m2_x,reanalysis_relative_humidity_percent_x,reanalysis_sat_precip_amt_mm_x,reanalysis_specific_humidity_g_per_kg_x,reanalysis_tdtr_k_x,station_diur_temp_rng_c_x,station_precip_mm_x,total_cases
0,sj,1990,18,4/30/1990,12.42,292.414286,299.8,295.9,32.00,73.365714,12.42,14.012857,2.628571,6.900000,16.0,4
1,sj,1990,19,5/7/1990,22.82,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,6.371429,8.6,5
2,sj,1990,20,5/14/1990,34.54,295.434286,300.5,297.3,26.10,82.052857,34.54,16.848571,2.300000,6.485714,41.4,4
3,sj,1990,21,5/21/1990,15.36,295.310000,301.4,297.0,13.90,80.337143,15.36,16.672857,2.428571,6.771429,4.0,3
4,sj,1990,22,5/28/1990,7.52,295.821429,301.9,297.5,12.20,80.460000,7.52,17.210000,3.014286,9.371429,5.8,6


In [41]:
sj_new.shape

(930, 16)

In [42]:
X = sj_new.drop(['city','total_cases','week_start_date'], axis=1)
y = sj_new['total_cases']

In [64]:
# Set seed for reproducibility
SEED = 123
# Split data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1397,random_state=SEED)


In [65]:
X_train.shape

(800, 13)

In [66]:
#using random forest to predict by hyperparameter tuning
# Import RandomForestRegressor

# Set seed for reproducibility
SEED_rf = 1
# Instantiate a random forests regressor 'rf'
rf = RandomForestRegressor(random_state= SEED_rf)

# Inspect rf' s hyperparameters
rf.get_params()

# Basic imports

# Define a grid of hyperparameter 'params_rf'
params_rf = {
'n_estimators': [50, 100, 200,300], #number of trees in the forest
'max_depth': [4, 6, 8,10],
'min_samples_leaf': [0.05,0.1, 0.2],
'max_features': ['log2','sqrt']
}
# Instantiate 'grid_rf'
grid_rf = GridSearchCV(estimator=rf,param_grid=params_rf,cv=5,scoring='neg_mean_squared_error',verbose=1,n_jobs=-1)

# Fit 'grid_rf' to the training set
grid_rf.fit(X_train, y_train)


# Extract best hyperparameters from 'grid_rf'
best_hyperparams = grid_rf.best_params_
print('Best hyerparameters:\n', best_hyperparams)


# Extract best model from 'grid_rf'
best_model = grid_rf.best_estimator_
# Predict the test set labels
y_pred = best_model.predict(X_test)
# Evaluate the test set RMSE
rmse_test = MAE(y_test, y_pred)
# Print the test set RMSE
print('Test set MAE of rf: {:.2f}'.format(rmse_test))


Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   25.8s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:   28.1s finished


Best hyerparameters:
 {'max_depth': 8, 'max_features': 'log2', 'min_samples_leaf': 0.05, 'n_estimators': 300}
Test set MAE of rf: 25.40


In [67]:
#parameters = {'alpha': [1e-10,1e-4,1e-1,0.5,0.8,1,5,10,20]}
lasso = Lasso(max_iter=10000, normalize=True)


lassocv = LassoCV(alphas=None, cv=10, max_iter=10000, normalize=True)
lassocv.fit(X_train, y_train)
lasso.set_params(alpha=lassocv.alpha_)
print("Alpha=", lassocv.alpha_)
lasso.fit(X_train, y_train)
print("mae = ",MAE(y_test, lasso.predict(X_test)))
print("best model coefficients:")
pd.Series(lasso.coef_, index=X.columns)

#y_pred = best_model.predict(X_test)
#Evaluate the test set RMSE
#rmse_test = MAE(y_test, y_pred)


Alpha= 0.04516361935310575
mae =  27.73958307023626
best model coefficients:


year                                      -1.956044
weekofyear                                 0.668003
precipitation_amt_mm_x                    -0.000000
reanalysis_dew_point_temp_k_x              0.000000
reanalysis_max_air_temp_k_x                3.908170
reanalysis_min_air_temp_k_x                0.000000
reanalysis_precip_amt_kg_per_m2_x          0.000000
reanalysis_relative_humidity_percent_x    -0.000000
reanalysis_sat_precip_amt_mm_x            -0.000000
reanalysis_specific_humidity_g_per_kg_x    0.000000
reanalysis_tdtr_k_x                       -0.000000
station_diur_temp_rng_c_x                 -0.000000
station_precip_mm_x                        0.000000
dtype: float64

In [103]:
#Gradient Boosting

gbt = GradientBoostingRegressor(n_estimators=300, max_depth=7, random_state=SEED)
# Fit 'gbt' to the training set
gbt.fit(X_train, y_train)
# Predict the test set labels
y_pred = gbt.predict(X_test)
# Evaluate the test set RMSE
mae_test = MAE(y_test, y_pred)

print(mae_test)


11.502959020297126


In [77]:
#taking coefficients from lasso only

lasso_var = ['year','weekofyear','reanalysis_max_air_temp_k_x']

gbt_l = GradientBoostingRegressor(n_estimators=300, max_depth=3, random_state=SEED)
# Fit 'gbt' to the training set
gbt_l.fit(X_train[lasso_var], y_train)
# Predict the test set labels
y_pred_l = gbt_l.predict(X_test[lasso_var])
# Evaluate the test set RMSE
mae_test_l = MAE(y_test, y_pred_l)

print(mae_test_l)

11.162255611363358


In [78]:
#this is great
#using this model to finally train the whole train data set and predict on the original test data set

lasso = Lasso(max_iter=10000, normalize=True)


lassocv = LassoCV(alphas=None, cv=10, max_iter=10000, normalize=True)
lassocv.fit(X, y)
lasso.set_params(alpha=lassocv.alpha_)
print("Alpha=", lassocv.alpha_)
lasso.fit(X, y)

print("best model coefficients:")
pd.Series(lasso.coef_, index=X.columns)


Alpha= 0.056740271885497184
best model coefficients:


year                                      -1.763699
weekofyear                                 0.686560
precipitation_amt_mm_x                    -0.000000
reanalysis_dew_point_temp_k_x              0.000000
reanalysis_max_air_temp_k_x                3.655960
reanalysis_min_air_temp_k_x                0.000000
reanalysis_precip_amt_kg_per_m2_x          0.000000
reanalysis_relative_humidity_percent_x     0.000000
reanalysis_sat_precip_amt_mm_x            -0.000000
reanalysis_specific_humidity_g_per_kg_x    0.000000
reanalysis_tdtr_k_x                       -0.000000
station_diur_temp_rng_c_x                  0.000000
station_precip_mm_x                        0.000000
dtype: float64

In [95]:
#importing the imputed test file now :))))
test_file = pd.read_csv(r'C:\Users\Aditya\Desktop\CS data competition\Imputed_test.csv')

test_file_sj = test_file[test_file['city']=='sj']


In [96]:
test_file.columns

Index(['city', 'year', 'weekofyear', 'week_start_date', 'ndvi_ne_x',
       'ndvi_nw_x', 'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm',
       'reanalysis_air_temp_k', 'reanalysis_avg_temp_k',
       'reanalysis_dew_point_temp_k', 'reanalysis_max_air_temp_k',
       'reanalysis_min_air_temp_k', 'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c_x', 'station_diur_temp_rng_c_x',
       'station_max_temp_c_x', 'station_min_temp_c_x', 'station_precip_mm_x',
       'ndvi_ne_y', 'ndvi_nw_y', 'station_avg_temp_c_y',
       'station_diur_temp_rng_c_y', 'station_max_temp_c_y',
       'station_min_temp_c_y', 'station_precip_mm_y'],
      dtype='object')

In [97]:
lasso_var = ['year','weekofyear','reanalysis_max_air_temp_k_x']

gbt_l = GradientBoostingRegressor(n_estimators=300, max_depth=3, random_state=SEED)
# Fit 'gbt' to the training set
gbt_l.fit(X[lasso_var], y)
# Predict the test set labels
y_pred_l = gbt_l.predict(test_file_sj[['year','weekofyear','reanalysis_max_air_temp_k']])
# Evaluate the test set RMSE
#mae_test_l = MAE(y_test, y_pred_l)

#print(mae_test_l)

In [98]:
y_pred_l

array([  5.41912336,   5.38925336,  10.86546831,  11.05475329,
        10.54617032,  12.60864993,  11.03833681,  51.17892343,
        16.58263088,  16.53246246,  19.39027976,  24.09725523,
        29.81222572,  33.9267336 ,  49.90117865,  71.21927794,
        51.00411322,  76.57480045, 121.85046377,  93.30784818,
        71.33262474,  71.63302533,  88.76901512,  74.18785695,
        81.19576358,  56.74952708,  46.56080044,  45.36603723,
        42.00451969,  41.16045087,  31.80354224,  20.59274889,
        17.09777319,  16.10616137,  17.85280114,  13.8493295 ,
        14.38003262,  11.17012622,   9.7492897 ,   7.0589892 ,
         7.52261355,   7.52261355,   6.00649489,   5.35283163,
         3.23740897,   3.76731381,   5.54765503,   2.38834134,
         3.07133894,   4.31508739,   5.41912336,   5.38925336,
         4.26938281,   4.76194928,   2.47132491,   6.17785293,
        12.60864993,  10.02242499,  10.58936261,  15.28423362,
        17.12987128,  23.01826389,  24.69466405,  30.40

In [99]:
test_file_sj['Output'] = y_pred_l

C:\Users\Aditya\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [100]:
test_file_sj

,city,year,weekofyear,week_start_date,ndvi_ne_x,ndvi_nw_x,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,station_min_temp_c_x,station_precip_mm_x,ndvi_ne_y,ndvi_nw_y,station_avg_temp_c_y,station_diur_temp_rng_c_y,station_max_temp_c_y,station_min_temp_c_y,station_precip_mm_y,Output
0,sj,2008,18,2008-04-29,-0.018900,-0.018900,0.102729,0.091200,78.60,298.492857,...,21.7,75.2,-0.005257,-0.003100,27.142857,6.188571,32.66,22.68,48.120,5.419123
1,sj,2008,19,2008-05-06,-0.018000,-0.012400,0.082043,0.072314,12.56,298.475714,...,22.2,34.3,0.017353,0.035593,26.382857,5.382857,30.00,22.66,40.660,5.389253
2,sj,2008,20,2008-05-13,-0.001500,0.051510,0.151083,0.091529,3.66,299.455714,...,22.8,3.0,-0.017763,0.051510,27.388571,5.885714,31.80,23.10,33.980,10.865468
3,sj,2008,21,2008-05-20,0.108304,-0.019867,0.124329,0.125686,0.00,299.690000,...,24.4,0.3,0.108304,0.039042,27.197143,5.982857,31.44,23.12,44.980,11.054753
4,sj,2008,22,2008-05-27,0.056800,0.039833,0.062267,0.075914,0.76,299.780000,...,23.3,84.1,0.036521,0.052783,27.725714,6.042857,31.90,24.00,50.940,10.546170
5,sj,2008,23,2008-06-03,-0.044000,-0.030467,0.132000,0.083529,71.17,299.768571,...,25.0,27.7,-0.004080,0.017845,28.097143,6.068571,32.78,23.76,40.420,12.608650
6,sj,2008,24,2008-06-10,-0.044300,-0.024925,0.132271,0.159157,48.99,300.062857,...,23.3,91.7,0.020217,0.051137,28.268571,6.485714,32.66,24.00,79.680,11.038337
7,sj,2008,25,2008-06-17,0.062308,0.082150,0.144371,0.116729,30.81,300.484286,...,24.4,0.3,0.062308,0.061027,28.497143,6.408571,33.44,24.08,50.380,51.178923
8,sj,2008,26,2008-06-24,0.010800,0.049900,0.100571,0.117329,8.02,300.601429,...,23.9,28.7,0.011275,0.032805,28.665714,6.568571,33.32,24.00,30.080,16.582631
9,sj,2008,27,2008-07-01,0.072667,0.106660,0.155429,0.164900,17.52,300.497143,...,25.0,2.9,-0.005597,0.038150,28.877143,6.011429,32.66,24.88,18.580,16.532462


In [101]:
test_file_sj.to_csv(r'C:\Users\Aditya\Desktop\CS data competition\Predictions_sj.csv', index = False)